# Size a HPP plant based on a simplified hpp model

This **static** notebook illustrates how to solve hybrid power plant sizing optimization in a specific location based on preselected turbine type.

**To execute this setup a server is required as it relies on paralle evaluation of the model.**

Sizing a hybrid power plant consists on designing the following parameters:

### Design Variables

**Wind Plant design:**

- Number of wind turbines in the wind plant [-] (`Nwt`)
- Wind power installation density [MW/km2] (`wind_MW_per_km2`): This parameter controls how closely spaced are the turbines, which in turns affect how much wake losses are present.

**PV Plant design:**

- Solar plant power capacity [MW] (`solar_MW`)

**Battery Storage design:**

- Battery power [MW] (`b_P`)
- Battery energy capacity in hours [MWh] (`b_E_h `): Battery storage capacity in hours of full battery power (`b_E = b_E_h * b_P `). 
- Cost of battery power fluctuations in peak price ratio [-] (`cost_of_batt_degr`): This parameter controls how much penalty is given to do ramps in battery power in the HPP operation.


### Possible objective functions

- LCOE: Levelized cost of energy.
- IRR: Internal rate of return. It is not defined for projects that produce negative Net present values. Hydesigb returns IRR = 0 if the NPV < 0. Nevertheless, optimizations can be problematic for sites without a clear case. 
- NPV/CAPEX: Net present value over total CAPEX. A good proxy variable, that will produce optimal sites with the optimal IRR, but that is defined on sites with negative NPV.

The available variables for optimiaztion are:

```
 'NPV_over_CAPEX',
 'NPV [MEuro]',
 'IRR',
 'LCOE [Euro/MWh]',
 'CAPEX [MEuro]',
 'OPEX [MEuro]',
 'penalty lifetime [MEuro]',
 ```
 

In [1]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git    

In [2]:
import pandas as pd
from hydesign.EGO_surrogate_based_optimization import EGO_path
from hydesign.examples import examples_filepath

# Optimize using EGO

In [3]:
examples_sites = pd.read_csv(f'{examples_filepath}examples_sites.csv', index_col=0)
examples_sites

,case,name,longitude,latitude,altitude,input_ts_fn,sim_pars_fn,price_fn,price_col
0,India,Indian_site_good_wind,77.500226,8.334294,679.803454,India/input_ts_Indian_site_good_wind.csv,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price
1,India,Indian_site_good_solar,68.542204,23.542099,29.883557,India/input_ts_Indian_site_good_solar.csv,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price
2,India,Indian_site_bad_solar_bad_wind,77.916878,17.292316,627.424643,India/input_ts_Indian_site_bad_solar_bad_wind.csv,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price
3,Europe,France_good_solar,4.229736,44.422011,204.000000,Europe/input_ts_France_good_solar.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R
4,Europe,France_good_wind,-0.864258,48.744116,302.000000,Europe/input_ts_France_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R
5,Europe,France_bad_solar_n_wind,2.167969,47.428087,140.000000,Europe/input_ts_France_bad_solar_n_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R
6,Europe,Germany_bad_solar_n_wind,10.766602,49.310798,442.000000,Europe/input_ts_Germany_bad_solar_n_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DE_ME
7,Europe,Germany_good_wind,7.873535,53.287111,5.000000,Europe/input_ts_Germany_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DE_NW
8,Europe,Denmark_good_solar,11.813965,55.397760,42.000000,Europe/input_ts_Denmark_good_solar.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DK_E
9,Europe,Denmark_good_wind,8.594398,56.227322,85.000000,Europe/input_ts_Denmark_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DK_W


In [4]:
EGO_simple = f'{EGO_path}EGO_surrogate_based_optimization_simple_hpp.py'

In [5]:
%run $EGO_simple --help

usage: EGO_surrogate_based_optimization_simple_hpp.py [-h] [--example EXAMPLE]
                                                      [--name NAME]
                                                      [--longitude LONGITUDE]
                                                      [--latitude LATITUDE]
                                                      [--altitude ALTITUDE]
                                                      [--input_ts_fn INPUT_TS_FN]
                                                      [--sim_pars_fn SIM_PARS_FN]
                                                      [--opt_var OPT_VAR]
                                                      [--rotor_diameter_m ROTOR_DIAMETER_M]
                                                      [--hub_height_m HUB_HEIGHT_M]
                                                      [--wt_rated_power_MW WT_RATED_POWER_MW]
                                                      [--surface_tilt_deg SURFACE_TILT_DEG]
                       

#### 

In [6]:
%run $EGO_simple \
    --example 4 \
    --opt_var "NPV_over_CAPEX"\
    --rotor_diameter_m 200\
    --hub_height_m 120\
    --wt_rated_power_MW 5\
    --surface_tilt_deg 15\
    --surface_azimuth_deg 180\
    --DC_AC_ratio 1.0\
    --num_batteries 2\
    --weeks_per_season_per_year 4 \
    --n_procs  31\
    --n_doe 124\
    --n_clusters 31\
    --n_seed 0\
    --max_iter 5\
    --final_design_fn 'hydesign_simple_design_0.csv'


# --weeks_per_season_per_year None \


Selected example site:
---------------------------------------------------
case                                         Europe
name                               France_good_wind
longitude                                 -0.864258
latitude                                  48.744116
altitude                                      302.0
input_ts_fn    Europe/input_ts_France_good_wind.csv
sim_pars_fn                     Europe/hpp_pars.yml
price_fn                   Europe/2030-EL_PRICE.csv
price_col                                      FR_R
Name: 4, dtype: object




Sizing a HPP plant at France_good_wind:


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302.0

rotor_diameter_m = 200.0
hub_height_m = 120.0
wt_rated_power_MW = 5.0
surface_tilt_deg = 15.0
surface_azimuth_deg = 180.0
DC_AC_ratio = 1.0


Selected input time series based on 4 weeks per season are stored in ./input_ts_sel.csv



Initial 124 simulations took 0.58

In [7]:
%run $EGO_simple \
    --example 4 \
    --opt_var "LCOE [Euro/MWh]"\
    --rotor_diameter_m 200\
    --hub_height_m 120\
    --wt_rated_power_MW 5\
    --surface_tilt_deg 15\
    --surface_azimuth_deg 180\
    --DC_AC_ratio 1.0\
    --num_batteries 2\
    --weeks_per_season_per_year 4 \
    --n_procs  31\
    --n_doe 124\
    --n_clusters 31\
    --n_seed 0\
    --max_iter 5\
    --final_design_fn 'hydesign_simple_design_1.csv'


Selected example site:
---------------------------------------------------
case                                         Europe
name                               France_good_wind
longitude                                 -0.864258
latitude                                  48.744116
altitude                                      302.0
input_ts_fn    Europe/input_ts_France_good_wind.csv
sim_pars_fn                     Europe/hpp_pars.yml
price_fn                   Europe/2030-EL_PRICE.csv
price_col                                      FR_R
Name: 4, dtype: object




Sizing a HPP plant at France_good_wind:


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302.0

rotor_diameter_m = 200.0
hub_height_m = 120.0
wt_rated_power_MW = 5.0
surface_tilt_deg = 15.0
surface_azimuth_deg = 180.0
DC_AC_ratio = 1.0


Selected input time series based on 4 weeks per season are stored in ./input_ts_sel.csv



Initial 124 simulations took 0.6 

# Size a HPP plant based on a detailed hpp model

This **static** notebook illustrates how to solve hybrid power plant sizing optimization in a specific location based on preselected turbine type.

**To execute this setup a server is required as it relies on paralle evaluation of the model.**

Sizing a hybrid power plant consists on designing the following parameters:

**Wind Turbine design:**

1. Clearance [m] (`clearance`): Height from the ground to rotor tip at lowest posstion. This parameter controls the wind turbine hub height given a rotor radius: `clearance =  HH - R`. 
2. Specific power of the wind turbine [MW/m2] (`sp`): Defined as `sp = p_rated / ( pi * R^2 )`, it controls the turbine rotor size for a given rated power. Turbines with lower specific power produce more power at lower wind speeds, but are more expensive.
3. Rated power of the wind turbine [MW] (`p_rated`)

**Wind Plant design:**

4. Number of wind turbines in the wind plant [-] (`Nwt`)
5. Wind power installation density [MW/km2] (`wind_MW_per_km2`): This parameter controls how closely spaced are the turbines, which in turns affect how much wake losses are present.

**PV Plant design:**

6. Solar plant power capacity [MW] (`solar_MW`)
7. Surface tilt [deg] (`surface_tilt`)
8. Surface azimuth [deg] (`surface_azimuth`)
9. DC-AC ratio [-] (`solar_DCAC`): This parameter controls how much over-planting of PV (in DC power) is connected to the inverters. It is common practice in PV design to have `solar_DCAC = 1.5`. 

**Battery Storage design:**

10. Battery power [MW] (`b_P`)
11. Battery energy capacity in hours [MWh] (`b_E_h `): Battery storage capacity in hours of full battery power (`b_E = b_E_h * b_P `). 
12. Cost of battery power fluctuations in peak price ratio [-] (`cost_of_batt_degr`): This parameter controls how much penalty is given to do ramps in battery power in the HPP operation.


### Possible objective functions

- LCOE: Levelized cost of energy.
- IRR: Internal rate of return. It is not defined for projects that produce negative Net present values. Hydesigb returns IRR = 0 if the NPV < 0. Nevertheless, optimizations can be problematic for sites without a clear case. 
- NPV/CAPEX: Net present value over total CAPEX. A good proxy variable, that will produce optimal sites with the optimal IRR, but that is defined on sites with negative NPV.

The available variables for optimiaztion are:

```
 'NPV_over_CAPEX',
 'NPV [MEuro]',
 'IRR',
 'LCOE [Euro/MWh]',
 'CAPEX [MEuro]',
 'OPEX [MEuro]',
 'penalty lifetime [MEuro]',
 ```
 

In [8]:
EGO_full = f'{EGO_path}EGO_surrogate_based_optimization.py'

In [9]:
%run $EGO_full --help

usage: EGO_surrogate_based_optimization.py [-h] [--example EXAMPLE]
                                           [--name NAME]
                                           [--longitude LONGITUDE]
                                           [--latitude LATITUDE]
                                           [--altitude ALTITUDE]
                                           [--input_ts_fn INPUT_TS_FN]
                                           [--sim_pars_fn SIM_PARS_FN]
                                           [--opt_var OPT_VAR]
                                           [--num_batteries NUM_BATTERIES]
                                           [--weeks_per_season_per_year WEEKS_PER_SEASON_PER_YEAR]
                                           [--n_procs N_PROCS] [--n_doe N_DOE]
                                           [--n_clusters N_CLUSTERS]
                                           [--n_seed N_SEED]
                                           [--max_iter MAX_ITER]
                         

In [10]:
%run $EGO_full \
    --example 4 \
    --opt_var "NPV_over_CAPEX"\
    --num_batteries 2\
    --weeks_per_season_per_year 4 \
    --n_procs  31\
    --n_doe 124\
    --n_clusters 31\
    --n_seed 0\
    --max_iter 5\
    --final_design_fn 'hydesign_design_full_0.csv'


Selected example site:
---------------------------------------------------
case                                         Europe
name                               France_good_wind
longitude                                 -0.864258
latitude                                  48.744116
altitude                                      302.0
input_ts_fn    Europe/input_ts_France_good_wind.csv
sim_pars_fn                     Europe/hpp_pars.yml
price_fn                   Europe/2030-EL_PRICE.csv
price_col                                      FR_R
Name: 4, dtype: object




Sizing a HPP plant at France_good_wind:


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302.0


Selected input time series based on 4 weeks per season are stored in ./input_ts_sel.csv



Initial 124 simulations took 0.59 minutes

Update sm and extract candidate points took 0.12 minutes
Check-optimal candidates: new 55 simulations took 0.31 minutes
  rel_yopt_

In [11]:
%run $EGO_full \
    --example 4 \
    --opt_var "LCOE [Euro/MWh]"\
    --num_batteries 2\
    --weeks_per_season_per_year 4 \
    --n_procs  31\
    --n_doe 124\
    --n_clusters 31\
    --n_seed 0\
    --max_iter 5\
    --final_design_fn 'hydesign_design_full_1.csv'


Selected example site:
---------------------------------------------------
case                                         Europe
name                               France_good_wind
longitude                                 -0.864258
latitude                                  48.744116
altitude                                      302.0
input_ts_fn    Europe/input_ts_France_good_wind.csv
sim_pars_fn                     Europe/hpp_pars.yml
price_fn                   Europe/2030-EL_PRICE.csv
price_col                                      FR_R
Name: 4, dtype: object




Sizing a HPP plant at France_good_wind:


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302.0


Selected input time series based on 4 weeks per season are stored in ./input_ts_sel.csv



Initial 124 simulations took 0.59 minutes

Update sm and extract candidate points took 0.11 minutes
Check-optimal candidates: new 41 simulations took 0.3 minutes
  rel_yopt_c